# **Workshop Demo**: Introduction to Machine Learning in Finance

- **Objectives**


# 1. Price Forecasting

In [ ]:
# Hassan TODO
print("hello world")

# 2. Risk Forecasting

In [ ]:
# Nico TODO
print("hello")

# 3. Deep Learning in Option Pricing

**What is Deep Learning**

- [Deep Learning](https://www.ibm.com/topics/deep-learning) is a subset of machine learning that uses multilayered neural networks to simulate the complex decision-making power of the human brain.

- Main difference between deep learning and machine learning is the structure of the underlying neural network architecture. "Nondeep" traditional ML models use simple neural networks with one or two computational layers. Deep learning models use neural networks with one or two compuetational layers. Deep learning models use three or more layers (typically hundreds or thousands of layers) to train the models.

- Consists of multiple layers of interconnected notes, each building on previous to refine and optimize the prediction -> forward propogation. The `IN` and `OUT` layers of a deep neural network are called **visible** layers

- Another process, backward propagation, is used to calculate errors in predictions and then adjust the weights and biases of between the node layers accordingly. The common approach to do this is via `gradient descent`.

- Common deep learning neural networks: 
    - **CNNs** (convolution neural networks): computer vision and image classification applications.
    - **RNNs** (recurrent neural networks): natural language and speech recognition.

**What is Option Pricing**
- Formal definition of an [option](https://corporatefinanceinstitute.com/resources/derivatives/option-pricing-models/) states that it is a type of contract between two parties that provides one party the right, but **not** the oblidation, to buy or sell the underlying asset at a predetermined price before or at expirations day.

    - Call option = buying
    - Put option = selling

- Buying or selling an option comes with a price called the option's premium. Buyer of an option pay the premium and sellers receive the premium

- Factors determining the value of an option:

    - Current stock price
    - Intrinsic value
    - Time to expiration
    - Volatility
    - Interest rates
    - Cash dividends paid

(ie. price of a stock rises -> likely price of call option will rise and the price of a put option will fall, vice versa)

- Example pricing models: 

    - Binomial Model
    - Trinomial Model
    - Black-Scholes Model
    - Monte-Carlo Simulation
    


**Black-Scholes (1973)**

- One of the most commonly used formula to price call and put options, there are many variations of this formula.
- Model was first discovered in 1973 by economists Fischer black and Myron Scholes, developed mainly for pricing European options on stocks.

- Some assumptions:
    - Stock Price Distribution : continuously compounded returns on the stock are normally distributed and independent over time, volatility of continuously compounded returns is known and constant, future dividends are known (as a dollar amount/fixed dividend yield)
    - Economic Environment : risk-free rate is known and constant, no transaction costs or taxes, possible to short-sell with no cost and to borrow at the risk-free rate

**Example Black-Scholes Code**

- Code below is from [Sanskar](https://github.com/Sanskar02/OPTION_PRICING_MODEL)
- Another example model can be found [here](https://srdas.github.io/Papers/BlackScholesNN.pdf)

**Generating Random Values to Simulate a Stock Option**

In [ ]:
# generate random Gaussian values
from random import seed
from random import gauss

# seed random number generator
seed(1)


# generting Stock Price(S) vector
list1 =[]
# Generates 10000 simulated stock prices
for i in range(10000):
    # Stock prices assumed to drawn from normal distribution with mean of 250 and standard deviation of 50.11
    k = gauss(250,50.11)
    # Rounds up generated value to two decimal places
    list1.append(round(k,2))
    

# generating Strike Price(K) vector
list2 =[]
for i in range(10000):
    k = 0
    # Generate non-negative stricke prices with mean of 322.6 and standard deviation of 65.9
    k = abs(gauss(322.6,65.9))
    while round(k,2) >= list1[i]:
        k = abs(gauss(322.6,65.9))
    list2.append(round(k,2))
    

# generate maturity time vector
list3 =[]
for i in range(10000):
    k = gauss(541.4,111)
    # Rounds generated values to 4 decimal places and converts value from days to years
    list3.append(round(int(round(k,2))/365,4))
    

# generate dividend gain vector    
list4 =[]
for i in range(10000):
    k = gauss(1.5,0.31)
    list4.append(round(round(k,2)/100,6))
    
    
# generate risk free interst rate vector
list5 =[]
for i in range(10000):
    k = gauss(2.05,0.2)
    list5.append(round(round(k,2)/100,6))


# generate volatality vector
list6 =[]
for i in range(10000):
    k = gauss(30,10)
    list6.append(round(round(k,2)/100,4))

In [ ]:
def listOfTuples(l1, l2,l3,l4,l5,l6): 
    return list(map(lambda x, y,z,w,i,o:(x,y,z,w,i,o), l1, l2,l3,l4,l5,l6)) 

p = listOfTuples(list1,list2,list3,list4,list5,list6)

**Creating DataFrame**

In [ ]:
import pandas as pd
df = pd.DataFrame(p, columns=["S","K","T","q","r","sigma"])
df.head(50)

In [ ]:
import numpy as np
import scipy.stats as si
import sympy as sy
from sympy.stats import Normal, cdf
from sympy import init_printing
init_printing()

# Function to define the Black-Scholes model
def black_scholes_call_div(S, K, T, r, q, sigma):
    
    #S: spot price (Price of underlying asset)
    #K: strike price
    #T: time to maturity
    #r: interest rate
    #q: rate of continuous dividend paying asset 
    #sigma: volatility
    
    # Two intermediate variables used in Black-Scholes formula
    d1 = (np.log(S / K) + (r - q + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = (np.log(S / K) + (r - q - 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    
   
    call =  (S * np.exp(-q * T) * si.norm.cdf(d1, 0.0, 1.0) - K * np.exp(-r * T) * si.norm.cdf(d2, 0.0, 1.0))
    
    # Returning calculated call option prices
    return round(call,2)

In [ ]:
# Applting the Black-Scholes function to the randomly generated dataset
df['C'] = df.apply(lambda row : black_scholes_call_div(row['S'], row['K'], row['T'],row['r'],row['q'],row['sigma']),axis = 1)
df.head(50)

In [ ]:
# Saving the DataFrame for efficient reload
df.to_pickle("option_dataset")

# Creating new DataFrame object df1 that points to same objects as df
df1 = df.copy()

In [ ]:
# Normalising the roperty of linear homogeneity in the Black-Scholes model
# Model's output price scales linearly with scale of input
df1["S"] = df1["S"]/df1["K"]
df1["C"] = df1["C"]/df1["K"]

# Now database S represents ratio of spot price to strike price (S/K)
# C represents ratio of normalizes option price (C/K)
df1.head()

In [ ]:
# Removing column K as it is now implied in the dataset under columns S and C
df1.drop(columns = ['K'], inplace = True)
df1.head(50)

In [ ]:
X = df1.copy()
# Dropping columns C and sigma (option price and volatility) to initialse training dataset
X.drop(columns = ['C'], inplace = True)
X.drop(columns = ['sigma'], inplace = True)
X.info()

In [ ]:
# Defining y as normalized C (option price) from before for future reference
y = df1['C']


In [ ]:
# Imports for creation of synthetic datasets and for splitting arrays/DataFrames into random train and test subsets
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split

# Splits feature matrix X and target vector y into training and testing sets
# 80% of data will be used for training and remaining 20% will be used for testing
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state = 1)

# Now stores 80% of X dataframe
X_train

In [ ]:
# Stores 80% of y dataframe, ground truth labels that the model will learn to predict from data in X_train
y_train

In [ ]:
X_test

In [ ]:
y_test

In [ ]:

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

[TensorFlow](https://www.tensorflow.org/) framework is used for creating and deploying ML models. 

In [ ]:
# importing TensorFlow library to prepare custom activation functions
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, LeakyReLU
from tensorflow.keras import backend as K

# Defining a custom activation function
def custom_activation(x):
    return K.exp(x)


**Building the ANN(artificial neural network) Model**

Three activation layers are added:

- LeakyRELU, function is h(x) = x if x ≥ 0, else h(x) = αx
- ELU (exponential linear unit). The function is h(x) = x if x ≥ 0, else h(x) = α(exp(x)-1).
- ReLU (rectified linear unit). THe function is h(x) = max(x, 0). The gradient of the function is zero in the region where x is negative, and the neuron is not active.

There is many more types of activation functions that can be added to a neural network. 

In [ ]:
nodes = 120
model = Sequential()

# Initial layer
# Adding LeakyRELU activation function
model.add(Dense(nodes, input_dim=X_train.shape[1]))
model.add(LeakyReLU())
# randomly deactivates 25% of nodes during training
model.add(Dropout(0.25))

# Hidden layers
model.add(Dense(nodes, activation='elu'))
model.add(Dropout(0.25))

model.add(Dense(nodes, activation='relu'))
model.add(Dropout(0.25))

model.add(Dense(nodes, activation='elu'))
model.add(Dropout(0.25))

# Output layer
model.add(Dense(1))
model.add(Activation(custom_activation))
          
# Compiles using mean squared error as loss function
# rmspropr optimizer, adapts learning rate during training based on magnitude of recent gradients
model.compile(loss='mse',optimizer='rmsprop')

In [ ]:
# Training the neural network model
# Model is learning from training dataset X_train as input and y_train as target output
# Data is split into mini-batches of 64 samples, updating node wirghts after processing each batch
# Will iterate over entire training dataset 20 times
# 10% of training data is held out as validation data for each epoch
model.fit(X_train, y_train, batch_size=64, epochs=20, validation_split=0.1, verbose=2)

In [ ]:
# Provides summary of neural network model's architecure:
    # Layer type and details, output shape parameters
model.summary()

In [ ]:
import math 
from sklearn.metrics import *
import matplotlib as mpl
import matplotlib.pyplot as plt

# Assess the accuracy of predictions generated by the model, comparing them to actual values
# y is the actual value and y_hat is the predicted
def CheckAccuracy(y,y_hat):
    stats = dict()

    stats['diff'] = y - y_hat
    
    
    #plot histogram to show the distribution of the differences between actual and predicted
    mpl.rcParams['agg.path.chunksize'] = 100000
    #figure(figsize=(14,10))
    plt.scatter(y, y_hat,color='black',linewidth=0.3,alpha=0.4, s=0.5)
    plt.xlabel('Actual Price',fontsize=20,fontname='Times New Roman')
    plt.ylabel('Predicted Price',fontsize=20,fontname='Times New Roman') 
    plt.show()
    
    #figure(figsize=(14,10))
    plt.hist(stats['diff'], bins=50,edgecolor='black',color='white')
    plt.xlabel('Diff')
    plt.ylabel('Density')
    plt.show()
    
    return stats

In [ ]:
# Predicted values from the training set
y_train_hat = model.predict(X_train)
#reduce dim (240000,1) -> (240000,) to match y_train's dim
y_train_hat = np.squeeze(y_train_hat)
CheckAccuracy(y_train, y_train_hat)